In [ ]:
import json
import pandas as pd
import numpy as np
import torch
import nltk
from nltk import TweetTokenizer
from nltk.corpus import stopwords
import string
import numpy as np
import json
nltk.download('stopwords')

In [ ]:
def get_file(filename):
    data = {}
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
        print("JSON data loaded successfully:")
    except Exception as e:
        print(e)
    return {"data": data}

In [ ]:
node_new_Twibot22 = get_file("Twibot-22/node_new.json")
label_new_Twibot22 = get_file("Twibot-22/label_new.json")

In [ ]:
point1 = node_new_Twibot22["data"]["t1498018021658431488"]
point1_author_id = point1["author_id"]

In [ ]:
# the tweet
print(point1)
# is the author at key "u" + author_id a bot or not? 
label_new_Twibot22["data"]["u"+str(point1_author_id)]

In [ ]:
# Count top-level keys in Twibot-22/node_new.json that start with 't'
try:
    # node_new_Twibot22 is expected to be a dict returned by get_file() earlier in this notebook
    data_obj = node_new_Twibot22 if not isinstance(node_new_Twibot22, dict) else node_new_Twibot22.get("data", node_new_Twibot22)
except NameError:
    # If the variable isn't defined (cell not run), load the file now
    print("node_new_Twibot22 not found in the notebook state - loading from file")
    node_new_Twibot22 = get_file("Twibot-22/node_new.json")
    data_obj = node_new_Twibot22.get("data", node_new_Twibot22)

# Ensure we have a mapping-like object
if not isinstance(data_obj, dict):
    print("Unexpected format: expected a JSON object/dict for node data.")
else:
    keys = [k for k in data_obj.keys() if isinstance(k, str) and k.startswith("t")]
    count = len(keys)
    print(f"Found {count} keys starting with 't' in Twibot-22/node_new.json")
    print(f"Scanned {len(data_obj)} top-level keys in the node object.")
    # Show up to 10 sample keys to verify
    print("Sample keys (up to 10):", keys[:10])

In [ ]:
# add a label to every tweet key
num_bot = 0 
num_human = 0
num_unlabeled = 0

for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    
    if "u"+str(cur_author_id) in label_new_Twibot22["data"]: 
        cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    else:
        num_unlabeled+= 1 # ids nonexistent in labels

    if (cur_author_label == "human"):
        num_human += 1
    if (cur_author_label == "bot"):
        num_bot += 1

    node_new_Twibot22["data"][cur_tweet_id]["author_label"] = cur_author_label

print(num_bot)
print(num_human)
print(num_unlabeled)

There are 8171 bot tweets, and 23057 human tweets


In [ ]:
# from pyspark.ml.feature import Tokenizer, CountVectorizer
# from pyspark.ml.classification import LogisticRegression

# tokenizer = Tokenizer(inputCol="text", outputCol="words")
# cv = CountVectorizer(inputCol="words", outputCol="features")
# lr = LogisticRegression(featuresCol="features", labelCol="label")

In [ ]:
# This code is for John Ryu because he is a meanie
complete_dict = {}

for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    
    if "u"+str(cur_author_id) not in label_new_Twibot22["data"]:
        continue
    
    cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    text = cur_tweet["text"]
    
    complete_dict[cur_tweet_id] = {
        "label": cur_author_label,
        "text": text
    }

with open("tweets_with_labels.json", "w", encoding="utf-8") as f:
    json.dump(complete_dict, f, ensure_ascii=False, indent=4)

In [ ]:
tknzr = TweetTokenizer(preserve_case =False)
stops = set(stopwords.words("english"))
stops.update(string.punctuation)
set_1 = {'…', '’', '...', '“', '”','‘'}
stops.update(set_1)


In [ ]:
print(tknzr.tokenize(node_new_Twibot22["data"][keys[0]]["text"]))
bot_dict = {}
human_dict = {}
for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    if "u"+str(cur_author_id) not in label_new_Twibot22["data"]: 
        continue
    cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    tokenized_tweet = tknzr.tokenize(cur_tweet["text"])
    filtered_tweet = [word for word in tokenized_tweet if word not in stops]
    if (cur_author_label == "human"):
        for token in filtered_tweet:
            if human_dict.get(token, 0) == 0:
                human_dict[token] = 1
            else:
                 human_dict[token] = human_dict[token]+1
    if (cur_author_label == "bot"):
        for token in filtered_tweet:
            if bot_dict.get(token, 0) == 0:
                bot_dict[token] = 1
            else:
                 bot_dict[token] = bot_dict[token]+1
sorted_bot = dict(sorted(bot_dict.items(), key=lambda item: item[1], reverse=True))
sorted_human = dict(sorted(human_dict.items(), key=lambda item: item[1], reverse=True))
print("Top 100 bot tokens:")
print(list(sorted_bot.items())[:100])
print("\nTop 100 human tokens:")
print(list(sorted_human.items())[:100])

    


In [ ]:
bot_lengths = np.array([])
human_lengths = np.array([])
for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    if "u"+str(cur_author_id) not in label_new_Twibot22["data"]: 
        continue
    cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    tokenized_tweet = tknzr.tokenize(cur_tweet["text"])
    if (cur_author_label == "human"):
        human_lengths = np.append(human_lengths, len(tokenized_tweet))
    if (cur_author_label == "bot"):
        bot_lengths = np.append(bot_lengths, len(tokenized_tweet))
def stats(arr):
    mean = np.mean(arr)
    std = np.std(arr)
    q2 = np.percentile(arr, 50) 
    minimum = np.min(arr)
    maximum = np.max(arr)

    print("Mean:", mean)
    print("Standard deviation:", std)
    print("Median (Q2):", q2)
    print("Minimum:", minimum)
    print("Maximum:", maximum)

print("Human tweet length stats:")
stats(human_lengths)
print("Bot tweet length stats:")
stats(bot_lengths)

In [ ]:
from nltk import bigrams
bot_dict = {}
human_dict = {}
for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    if "u"+str(cur_author_id) not in label_new_Twibot22["data"]: 
        continue
    cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    tokenized_tweet = tknzr.tokenize(cur_tweet["text"])
    filtered_tweet = [word for word in tokenized_tweet if word not in stops]
    filtered_tweet = bigrams(filtered_tweet)
    if (cur_author_label == "human"):
        for token in filtered_tweet:
            if human_dict.get(token, 0) == 0:
                human_dict[token] = 1
            else:
                 human_dict[token] = human_dict[token]+1
    if (cur_author_label == "bot"):
        for token in filtered_tweet:
            if bot_dict.get(token, 0) == 0:
                bot_dict[token] = 1
            else:
                 bot_dict[token] = bot_dict[token]+1
sorted_bot = dict(sorted(bot_dict.items(), key=lambda item: item[1], reverse=True))
sorted_human = dict(sorted(human_dict.items(), key=lambda item: item[1], reverse=True))
print("Top 100 bot tokens:")
print(list(sorted_bot.items())[:100])
print("\nTop 100 human tokens:")
print(list(sorted_human.items())[:100])

In [ ]:
from nltk import trigrams
bot_dict = {}
human_dict = {}
for cur_tweet_id in keys: 
    cur_tweet = node_new_Twibot22["data"][cur_tweet_id]
    cur_author_id = cur_tweet["author_id"]
    if "u"+str(cur_author_id) not in label_new_Twibot22["data"]: 
        continue
    cur_author_label = label_new_Twibot22["data"]["u"+str(cur_author_id)]
    tokenized_tweet = tknzr.tokenize(cur_tweet["text"])
    filtered_tweet = [word for word in tokenized_tweet if word not in stops]
    filtered_tweet = trigrams(filtered_tweet)
    if (cur_author_label == "human"):
        for token in filtered_tweet:
            if human_dict.get(token, 0) == 0:
                human_dict[token] = 1
            else:
                 human_dict[token] = human_dict[token]+1
    if (cur_author_label == "bot"):
        for token in filtered_tweet:
            if bot_dict.get(token, 0) == 0:
                bot_dict[token] = 1
            else:
                 bot_dict[token] = bot_dict[token]+1
sorted_bot = dict(sorted(bot_dict.items(), key=lambda item: item[1], reverse=True))
sorted_human = dict(sorted(human_dict.items(), key=lambda item: item[1], reverse=True))
print("Top 100 bot tokens:")
print(list(sorted_bot.items())[:100])
print("\nTop 100 human tokens:")
print(list(sorted_human.items())[:100])